### Randomized Design

The treatment and control units are selscted completely randomly. This technique is used primarily in the situation where there's very little opportunity to control variables, and where the volume and velocity of data is high enough that you are not worried about bias.

### Experiment

A main company goal of large grocery chain is to drive more customers to download their mobile app, and register for their loyality program.One way of driving customers to download the app, is vai link on the company website. According to page viewing statistics show that only 10% of people who visit the main page are clicling through to visit the page that describe the app, and loyality program.

You have asked expermnet whether changing the button of the page link to picture would improve the click throgh rates of vistors who visit the page thta describe the app and loyal to program.

#### Brainstrom control variables   ?????????????????????

1. Types of people vistingb website( Sociaeconomic stading and demographic profile etc)

2. Site visits only count once( frequency of visits)

3. Membership already have of not

### Sample Size

Units of Diversion  === IP Adress

Population          === Visitors without an account

Duration            === 1 Week

Size                === 1/3 treatment and 2/3 control


In randomized design experiment units are assigned randomly to treatment and control group

In [45]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from statsmodels.stats.power import TTestIndPower
import statsmodels.api as sm
from pandas import get_dummies
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from scipy.stats import linregress
from sklearn.linear_model import LogisticRegression
import ggplot
from pandas.lib import Timestamp
from scipy import stats

In [46]:
# importing the data in dataframe from csv
df = pd.read_csv("grocerywebsiteabtestdata (1).csv")

In [47]:
df.head()

,RecordID,IP Address,LoggedInFlag,ServerID,VisitPageFlag
0,1,39.13.114.2,1,2,0
1,2,13.3.25.8,1,1,0
2,3,247.8.211.8,1,1,0
3,4,124.8.220.3,0,3,0
4,5,60.10.192.7,0,2,0


### Understading the data

1. RecordID is just the identifier of the row data
2. The IP Adress is the address of the user who is visiting the website.
3. The loggedin Flag is 1 if the user has an account is logged in.
4. The serverID is one of the three servers that the user was routed through( ServerID 2 and 3 represent the control group and      serverID 1 is treatment group)
5. VisitPageFlag the user clicked on the page with information about the loyality program

In [48]:
df.dtypes

RecordID          int64
IP Address       object
LoggedInFlag      int64
ServerID          int64
VisitPageFlag     int64
dtype: object

In [49]:
df.shape

(184588, 5)

### Removing the already loggedin user data as they are already the customer 

In [50]:
df.drop(df[df['LoggedInFlag'] == 1].index, inplace = True) 

In [51]:
df.shape

(91619, 5)

In [52]:
df.head()

,RecordID,IP Address,LoggedInFlag,ServerID,VisitPageFlag
3,4,124.8.220.3,0,3,0
4,5,60.10.192.7,0,2,0
7,8,97.6.126.6,0,3,1
10,11,32.6.213.1,0,2,0
12,13,188.13.62.2,0,3,0


### Removing if any duplicates ip addresses are present in the dataset only once should be considered 

In [53]:
New = df.drop_duplicates("IP Address")
New.shape

(49513, 5)

In [54]:
df1 =[]
for length in New.ServerID:
    if length == 1:
        df1.append("Treatment")
    else:
        df1.append("Control")

In [55]:
df1[0:5]

['Control', 'Control', 'Control', 'Control', 'Control']

In [56]:
New["Group"] = df1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
New.head()

,RecordID,IP Address,LoggedInFlag,ServerID,VisitPageFlag,Group
3,4,124.8.220.3,0,3,0,Control
4,5,60.10.192.7,0,2,0,Control
7,8,97.6.126.6,0,3,1,Control
10,11,32.6.213.1,0,2,0,Control
12,13,188.13.62.2,0,3,0,Control


### Data is prepared for Analysis

We have the data indicating the group the visitor is in, Whether or not they visited the page by clikcing on the link. We don't have any people who have logged in and we have only 1 row per visitor

### Test of means analysis

Statitical calculation of wheter the mean values of the treatment and control groups are the same

In other words there is any statistical diffrence in result of not

1. First point to look into the target variable average in treatment and control group

2. Test of means analysis include T-Test which gives P- value( liklihood the actual diffrence b/w the means is 0)

3. Generally, if the P-value is below 0.05, indicating a confidence interval of 95% the diffrence in mean is considered statistically significant

In [57]:
New.VisitPageFlag.mean()

0.11657544483267021

In [80]:
New.groupby('Group').VisitPageFlag.mean()

Group
Control      0.105460
Treatment    0.138729
Name: VisitPageFlag, dtype: float64

In [77]:
New.groupby(['Group','VisitPageFlag']).Group.count()

Group      VisitPageFlag
Control    0                29493
           1                 3477
Treatment  0                14248
           1                 2295
Name: Group, dtype: int64